In [14]:
import pandas as pd
import yfinance as yf
import talib as ta
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from backtesting import Strategy, Backtest
from backtesting.lib import crossover, TrailingStrategy, resample_apply

In [7]:
pd.options.mode.chained_assignment = None

In [30]:
start = dt.datetime(2018,1,1)
end = dt.datetime.now()
asset = str(input('Look on Yahoo Finance for ticker.'
                  'Asset to backtest: '))
data = yf.download(asset, start=start, end=end)
data

Look on Yahoo Finance for ticker.Asset to backtest: TSLA
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,20.799999,21.474001,20.733334,21.368668,21.368668,65283000
2018-01-03,21.400000,21.683332,21.036667,21.150000,21.150000,67822500
2018-01-04,20.858000,21.236668,20.378668,20.974667,20.974667,149194500
2018-01-05,21.108000,21.149332,20.799999,21.105333,21.105333,68868000
2018-01-08,21.066668,22.468000,21.033333,22.427334,22.427334,147891000
...,...,...,...,...,...,...
2023-12-04,235.750000,239.369995,233.289993,235.580002,235.580002,104099800
2023-12-05,233.869995,246.660004,233.699997,238.720001,238.720001,137971100
2023-12-06,242.919998,246.570007,239.169998,239.369995,239.369995,126436200


In [33]:
class MyStrategy(TrailingStrategy):
    fast = 50
    slow = 200
    upper_bound = 70
    lower_bound = 30

    def init(self):
        super().init()
        self.set_trailing_sl(3)
              
        close = self.data.Close
        self.Hourly_sma1 = resample_apply('H',ta.SMA, close, self.fast)
        self.Hourly_sma2 = resample_apply('H',ta.SMA, close, self.slow)
        self.daily_rsi = self.I(ta.RSI, close, 14)
        self.hourly_rsi = resample_apply('H', ta.RSI, close, 20)
        self.Long_tp = self.daily_rsi > self.upper_bound
        self.Short_tp = self.daily_rsi < self.lower_bound

    def next(self):
        super().next()
        close = self.data.Close[-1]

        if crossover(self.Hourly_sma1, self.Hourly_sma2) and self.hourly_rsi < self.upper_bound and not self.position:
                self.buy(tp = self.Long_tp)

        elif crossover(self.Hourly_sma2, self.Hourly_sma1) and self.hourly_rsi > self.lower_bound and not self.position:
                self.sell(size=0.75,
                          tp = self.Short_tp)

In [34]:
bt = Backtest(data, MyStrategy, cash=10000, commission=0.02, exclusive_orders=True)
output= bt.run()
print(output)
bt.plot()

Start                     2018-01-02 00:00:00
End                       2023-12-08 00:00:00
Duration                   2166 days 00:00:00
Exposure Time [%]                   14.448161
Equity Final [$]                 15979.570277
Equity Peak [$]                  17607.922358
Return [%]                          59.795703
Buy & Hold Return [%]             1041.109969
Return (Ann.) [%]                    8.221441
Volatility (Ann.) [%]               15.329643
Sharpe Ratio                          0.53631
Sortino Ratio                        0.959221
Calmar Ratio                         0.548037
Max. Drawdown [%]                  -15.001629
Avg. Drawdown [%]                   -4.837762
Max. Drawdown Duration      868 days 00:00:00
Avg. Drawdown Duration      113 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                      53.558894
Worst Trade [%]                     -9.349152
Avg. Trade [%]                    

Row(id='5486', ...)